In [29]:
import pandas as pd
import numpy as np
import os
import joblib as joblib
import pickle

from tqdm.auto import tqdm
from datetime import datetime
from scipy.stats import zscore

# from ReMASTER.system import get_data_dir

tqdm.pandas()

In [30]:
# file_path = '/Users/areejmirani/Desktop/ReMASTER/data/NQ_5Years_8_11_2024.csv'
file_path = 'C:\\Users\\amirani\\ReMASTER\\NQ_5Years_8_11_2024.csv'

# Initialize an empty DataFrame for the combined data
# data = pd.DataFrame()
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe to inspect its structure
data.head()

,Time,Open,High,Low,Close,Volume
0,8/11/2019 23:05,9073.25,9098.50,9073.00,9092.50,1758
1,8/11/2019 23:10,9093.25,9095.50,9089.75,9092.75,438
2,8/11/2019 23:15,9093.00,9096.25,9088.00,9089.75,590
3,8/11/2019 23:20,9090.25,9090.25,9086.00,9087.00,278
4,8/11/2019 23:25,9086.75,9088.25,9079.75,9083.75,711


In [31]:
# 1. Convert 'Date' to datetime format
data['Date'] = pd.to_datetime(data['Time'])

# Set 'Date' as the index for time-based operations
data.set_index('Date', inplace=True)

# Verify the structure (no need to drop 'Time', as it's part of the index now)
data.head()

,Time,Open,High,Low,Close,Volume
Date,,,,,,
2019-08-11 23:05:00,8/11/2019 23:05,9073.25,9098.50,9073.00,9092.50,1758
2019-08-11 23:10:00,8/11/2019 23:10,9093.25,9095.50,9089.75,9092.75,438
2019-08-11 23:15:00,8/11/2019 23:15,9093.00,9096.25,9088.00,9089.75,590
2019-08-11 23:20:00,8/11/2019 23:20,9090.25,9090.25,9086.00,9087.00,278
2019-08-11 23:25:00,8/11/2019 23:25,9086.75,9088.25,9079.75,9083.75,711


## updated method to ensure this set fits the csi300 form with alpha158 form etc.

# reformat to fit multi-index and get the specified features that csi300 has

this is what csi300's formulas were:

MultiIndex([('feature',                              '($close-$open)/$open'),
            ('feature',                                '($high-$low)/$open'),
            ('feature',                 '($close-$open)/($high-$low+1e-12)'),
            ('feature',              '($high-Greater($open, $close))/$open'),
            ('feature', '($high-Greater($open, $close))/($high-$low+1e-12)'),
            ('feature',                  '(Less($open, $close)-$low)/$open'),
            ('feature',     '(Less($open, $close)-$low)/($high-$low+1e-12)'),
            ('feature',                       '(2*$close-$high-$low)/$open'),
            ('feature',          '(2*$close-$high-$low)/($high-$low+1e-12)'),
            ('feature',                                      '$open/$close'),
            ...
            ('feature',          'Mask(Std($amount,20)/$amount,'SH000906')'),
            ('feature',  'Mask(Mean($close/Ref($close,1)-1,30),'SH000906')'),
            ('feature',   'Mask(Std($close/Ref($close,1)-1,30),'SH000906')'),
            ('feature',         'Mask(Mean($amount,30)/$amount,'SH000906')'),
            ('feature',          'Mask(Std($amount,30)/$amount,'SH000906')'),
            ('feature',  'Mask(Mean($close/Ref($close,1)-1,60),'SH000906')'),
            ('feature',   'Mask(Std($close/Ref($close,1)-1,60),'SH000906')'),
            ('feature',         'Mask(Mean($amount,60)/$amount,'SH000906')'),
            ('feature',          'Mask(Std($amount,60)/$amount,'SH000906')'),
            (  'label',             'Ref($close, -5) / Ref($close, -1) - 1')],
           length=222)

In [32]:
# claude version


import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import linregress

def calculate_csi300_features(df, instrument_id='NQ100'):
    """Calculate features and reshape data into CSI300 format."""
    # Make a copy of the dataframe
    df = df.copy()

    # Create features dictionary
    features = {}

    # Price Change Ratios
    features['(close-open)/open'] = (df['Close'] - df['Open']) / df['Open']
    features['(high-low)/open'] = (df['High'] - df['Low']) / df['Open']
    features['(2*close-high-low)/open'] = (2 * df['Close'] - df['High'] - df['Low']) / df['Open']
    print("Price Change Ratios done")

    # Moving Averages and Standard Deviations (Volatility)
    features['Mean(close,5)/close'] = df['Close'].rolling(window=5).mean() / df['Close']
    features['Mean(close,10)/close'] = df['Close'].rolling(window=10).mean() / df['Close']
    features['Mean(close,20)/close'] = df['Close'].rolling(window=20).mean() / df['Close']
    features['Std(close,5)/close'] = df['Close'].rolling(window=5).std() / df['Close']
    features['Std(close,10)/close'] = df['Close'].rolling(window=10).std() / df['Close']
    features['Std(close,20)/close'] = df['Close'].rolling(window=20).std() / df['Close']
    print("Moving Averages and Standard Deviations done")

    # Relative Strength and Trends (Slope calculation)
    for window in [5, 10, 20]:
        features[f'Slope(close,{window})/close'] = df['Close'].rolling(window).apply(
            lambda x: linregress(range(len(x)), x).slope / x[-1] if len(x) == window else np.nan
        )
    print("Relative Strength and Trends done")

    # Volume-Weighted Measures
    features['VWAP/close'] = (df['Volume'] * df['Close']).cumsum() / df['Volume'].cumsum() / df['Close']
    features['Mean(volume,5)/(volume+1e-12)'] = df['Volume'].rolling(window=5).mean() / (df['Volume'] + 1e-12)
    features['Std(volume,5)/(volume+1e-12)'] = df['Volume'].rolling(window=5).std() / (df['Volume'] + 1e-12)
    print("Volume-Weighted Measures done")

    # Historical Close Ratios
    features['Ref(close,5)/close'] = df['Close'].shift(5) / df['Close']
    features['Ref(close,10)/close'] = df['Close'].shift(10) / df['Close']
    features['Ref(close,20)/close'] = df['Close'].shift(20) / df['Close']
    print("Historical Close Ratios done")

    # Correlation Measures
    features['Corr(close,log(volume+1),5)'] = df['Close'].rolling(window=5).corr(np.log(df['Volume'] + 1))
    features['Corr(close/Ref(close,1),log(volume/Ref(volume,1)+1),5)'] = (
        (df['Close'] / df['Close'].shift(1)).rolling(window=5)
        .corr(np.log(df['Volume'] / df['Volume'].shift(1) + 1))
    )
    print("Correlation Measures done")

    # Price Extremes and Ratios
    features['Max(high,5)/close'] = df['High'].rolling(window=5).max() / df['Close']
    features['Min(low,5)/close'] = df['Low'].rolling(window=5).min() / df['Close']
    print("Price Extremes and Ratios done")

    # Create DataFrame from features with the same index as input data
    feature_df = pd.DataFrame(features, index=df.index)

    # Add datetime (using the index date) and instrument columns
    feature_df['datetime'] = feature_df.index.date
    feature_df['instrument'] = instrument_id

    # Reorder columns to put datetime and instrument first
    cols = ['datetime', 'instrument'] + [col for col in feature_df.columns if col not in ['datetime', 'instrument']]
    feature_df = feature_df[cols]

    return feature_df

def process_single_instrument(data, instrument_id, date_ranges):
    """Process data for a single instrument."""
    # Calculate features
    processed_df = calculate_csi300_features(data, instrument_id)

    # Set datetime and instrument as index
    processed_df.set_index(['datetime', 'instrument'], inplace=True)

    # Split data based on date ranges
    train_data = processed_df[processed_df.index.get_level_values(0) <= date_ranges['train_end'].date()]
    valid_data = processed_df[
        (processed_df.index.get_level_values(0) >= date_ranges['valid_start'].date()) &
        (processed_df.index.get_level_values(0) <= date_ranges['valid_end'].date())
    ]
    test_data = processed_df[
        (processed_df.index.get_level_values(0) >= date_ranges['test_start'].date()) &
        (processed_df.index.get_level_values(0) <= date_ranges['test_end'].date())
    ]

    return train_data, valid_data, test_data

# Debug: Print input data structure
print("Input data structure:")
print(data.head())
print("\nInput data info:")
print(data.info())

# Define date ranges
date_ranges = {
    'train_end': datetime(2022, 3, 31),
    'valid_start': datetime(2022, 4, 1),
    'valid_end': datetime(2022, 6, 30),
    'test_start': datetime(2022, 7, 1),
    'test_end': datetime(2024, 12, 31)
}

# Process the data for NQ100
instrument_id = 'NQ100'
train_data, valid_data, test_data = process_single_instrument(data, instrument_id, date_ranges)

# Save the processed datasets
train_data.to_csv('train_data_2020_2022_NQ100.csv')
valid_data.to_csv('valid_data_2022_NQ100.csv')
test_data.to_csv('test_data_2022_2024_NQ100.csv')

# Print sample output to verify format
print("\nSample of training data:")
print(train_data.head())

# Print shape information
print("\nDataset shapes:")
print(f"Train data: {train_data.shape}")
print(f"Validation data: {valid_data.shape}")
print(f"Test data: {test_data.shape}")

Input data structure:
                                Time     Open     High      Low    Close  \
Date                                                                       
2019-08-11 23:05:00  8/11/2019 23:05  9073.25  9098.50  9073.00  9092.50   
2019-08-11 23:10:00  8/11/2019 23:10  9093.25  9095.50  9089.75  9092.75   
2019-08-11 23:15:00  8/11/2019 23:15  9093.00  9096.25  9088.00  9089.75   
2019-08-11 23:20:00  8/11/2019 23:20  9090.25  9090.25  9086.00  9087.00   
2019-08-11 23:25:00  8/11/2019 23:25  9086.75  9088.25  9079.75  9083.75   

                     Volume  
Date                         
2019-08-11 23:05:00    1758  
2019-08-11 23:10:00     438  
2019-08-11 23:15:00     590  
2019-08-11 23:20:00     278  
2019-08-11 23:25:00     711  

Input data info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 329458 entries, 2019-08-11 23:05:00 to 2024-08-09 16:55:00
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----

C:\Users\amirani\AppData\Local\Temp\5\ipykernel_24112\1886209016.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda x: linregress(range(len(x)), x).slope / x[-1] if len(x) == window else np.nan


Relative Strength and Trends done
Volume-Weighted Measures done
Historical Close Ratios done
Correlation Measures done
Price Extremes and Ratios done

Sample of training data:
                       (close-open)/open  (high-low)/open  \
datetime   instrument                                       
2019-08-11 NQ100                0.002122         0.002810   
           NQ100               -0.000055         0.000632   
           NQ100               -0.000357         0.000907   
           NQ100               -0.000358         0.000468   
           NQ100               -0.000330         0.000935   

                       (2*close-high-low)/open  Mean(close,5)/close  \
datetime   instrument                                                 
2019-08-11 NQ100                      0.001488                  NaN   
           NQ100                      0.000027                  NaN   
           NQ100                     -0.000522                  NaN   
           NQ100                     -0.0

# convert to .pkl

In [33]:
# 6. Save each set as .pkl files
# # locally
# train_path = '/Users/areejmirani/Desktop/ReMASTER/data/NQ100/NQ100_dl_train.pkl'
# valid_path = '/Users/areejmirani/Desktop/ReMASTER/data/NQ100/NQ100_dl_valid.pkl'
# test_path = '/Users/areejmirani/Desktop/ReMASTER/data/NQ100/NQ100_dl_test.pkl'

# # google drive for colab
# train_path = '/content/drive/My Drive/ECE 570/data/NQ100/NQ100_dl_train.pkl'
# valid_path = '/content/drive/My Drive/ECE 570/data/NQ100/NQ100_dl_valid.pkl'
# test_path = '/content/drive/My Drive/ECE 570/data/NQ100/NQ100_dl_test.pkl'

# onedrive when working at krach
train_path = 'C:\\Users\\amirani\\ReMASTER\\data\\NQ100\\NQ100_dl_train.pkl'
valid_path = 'C:\\Users\\amirani\\ReMASTER\\data\\NQ100\\NQ100_dl_valid.pkl'
test_path = 'C:\\Users\\amirani\\ReMASTER\\data\\NQ100\\NQ100_dl_test.pkl'

# Save train data
with open(train_path, 'wb') as f:
    pickle.dump(train_data, f)

# Save validation data
with open(valid_path, 'wb') as f:
    pickle.dump(valid_data, f)

# Save test data
with open(test_path, 'wb') as f:
    pickle.dump(test_data, f)

# Return the paths (optional)
train_path, valid_path, test_path

('C:\\Users\\amirani\\ReMASTER\\data\\NQ100\\NQ100_dl_train.pkl',
 'C:\\Users\\amirani\\ReMASTER\\data\\NQ100\\NQ100_dl_valid.pkl',
 'C:\\Users\\amirani\\ReMASTER\\data\\NQ100\\NQ100_dl_test.pkl')

## ensure pkl files have content 

In [34]:
# Function to load and test if data exists in each .pkl file
def test_data_in_pkl(file_path):
    try:
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
        # Check if the DataFrame is not empty
        if isinstance(data, pd.DataFrame) and not data.empty:
            print(f"Data exists in {file_path} and has {len(data)} rows and {len(data.columns)} columns.")
        else:
            print(f"File {file_path} is either empty or not a DataFrame.")
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

# Test each .pkl file
test_data_in_pkl(train_path)
test_data_in_pkl(valid_path)
test_data_in_pkl(test_path)



Data exists in C:\Users\amirani\ReMASTER\data\NQ100\NQ100_dl_train.pkl and has 171002 rows and 22 columns.
Data exists in C:\Users\amirani\ReMASTER\data\NQ100\NQ100_dl_valid.pkl and has 15987 rows and 22 columns.
Data exists in C:\Users\amirani\ReMASTER\data\NQ100\NQ100_dl_test.pkl and has 142469 rows and 22 columns.


## old implementation

In [35]:
# # Step 1: Convert 'Time' column to datetime format
# data['Time'] = pd.to_datetime(data['Time'], format='%m/%d/%Y %H:%M')

# # Step 2: Check for any NA values and drop columns with NA values (if any)
# data.dropna(axis=1, inplace=True)

# # Step 3: Perform robust daily Z-score normalization on each feature dimension
# # Extract date component for daily grouping
# data['Date'] = data['Time'].dt.date
# # Group by 'Date' and normalize 'Open', 'High', 'Low', 'Close', and 'Volume' columns
# feature_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
# data[feature_columns] = data.groupby('Date')[feature_columns].transform(zscore)

# # Step 4: Drop 5% of the most extreme values from the 'Close' column to reduce label outliers
# # Identify upper and lower 2.5% quantiles for 'Close'
# q_low, q_high = data['Close'].quantile([0.025, 0.975])
# data = data[(data['Close'] >= q_low) & (data['Close'] <= q_high)]

# # Step 5: Drop NA rows (if any remain after filtering) and reset index
# data.dropna(inplace=True)
# data.reset_index(drop=True, inplace=True)

# # Step 6: Save the cleaned data to a new CSV file
# # Create the directory if it doesn't exist
# save_path = 'new_data/reshaped_data.csv'
# os.makedirs(os.path.dirname(save_path), exist_ok=True)
# data.to_csv(save_path, index=False)

# # Display first few rows of the cleaned data to confirm
# data.head()

# print(data.columns)


Split data into train, valid, and test sets by datetime. Earlier years are in training, recent years in test. 

In [36]:
# import pandas as pd
# import pickle
# from sklearn.preprocessing import MinMaxScaler
# import numpy as np

# # Assuming 'data' is your original dataframe with columns like 'Date', 'Time', 'Open', 'High', 'Low', 'Close', etc.

# # 1. Combine Date and Time into a single 'datetime' column
# # Ensure that 'Date' is a string or datetime type and 'Time' is a string (e.g., "HH:MM:SS")

# # Convert 'Date' to datetime (if not already in datetime format)
# data['Date'] = pd.to_datetime(data['Date'])

# # Check the type of 'Time' column to handle it accordingly
# if data['Time'].dtype == 'datetime64[ns]':  # If Time is already in datetime64[ns]
#     data['Time'] = data['Time'].dt.time  # Extract the time part only
# else:  # If Time is a string like 'HH:MM:SS'
#     data['Time'] = pd.to_timedelta(data['Time'], errors='coerce')

# # Combine 'Date' and 'Time' into a single 'datetime' column
# data['datetime'] = data['Date'] + pd.to_timedelta(data['Time'].astype(str))

# # Drop the original 'Date' and 'Time' columns if no longer needed
# data.drop(columns=['Date', 'Time'], inplace=True)

# # 2. Feature Engineering
# data['close_open'] = (data['Close'] - data['Open']) / data['Open']
# data['high_low'] = (data['High'] - data['Low']) / data['Open']
# data['close_open_highlow'] = (data['Close'] - data['Open']) / (data['High'] - data['Low'] + 1e-12)
# data['high_max_open_close'] = (data['High'] - data[['Open', 'Close']].max(axis=1)) / data['Open']

# # 3. Handle NaN or infinite values
# # Replace NaN with the column mean, or you can use other strategies
# data.fillna(data.mean(), inplace=True)

# # Replace any infinity values with a large number or NaN
# data.replace([np.inf, -np.inf], np.nan, inplace=True)

# # Reapply fillna to handle any NaNs created by infinities
# data.fillna(data.mean(), inplace=True)

# # 4. Normalize the newly created features
# scaler = MinMaxScaler()

# # Select columns for scaling
# features = ['close_open', 'high_low', 'close_open_highlow', 'high_max_open_close']
# data[features] = scaler.fit_transform(data[features])

# # 5. Split the data into train, valid, and test
# train_size = int(0.7 * len(data))  # 70% for training
# valid_size = int(0.15 * len(data))  # 15% for validation
# test_size = len(data) - train_size - valid_size  # 15% for testing

# train_data = data[:train_size]
# valid_data = data[train_size:train_size + valid_size]
# test_data = data[train_size + valid_size:]

# # 6. Save each set as .pkl files
# train_path = '/Users/areejmirani/Desktop/ReMASTER/data/NQ100/NQ100_dl_train.pkl'
# valid_path = '/Users/areejmirani/Desktop/ReMASTER/data/NQ100/NQ100_dl_valid.pkl'
# test_path = '/Users/areejmirani/Desktop/ReMASTER/data/NQ100/NQ100_dl_test.pkl'

# # Save train data
# with open(train_path, 'wb') as f:
#     pickle.dump(train_data, f)

# # Save validation data
# with open(valid_path, 'wb') as f:
#     pickle.dump(valid_data, f)

# # Save test data
# with open(test_path, 'wb') as f:
#     pickle.dump(test_data, f)

# # Return the paths (optional)
# train_path, valid_path, test_path


Ensure data is in the .pkl files

In [37]:
# # Function to load and test if data exists in each .pkl file
# def test_data_in_pkl(file_path):
#     try:
#         with open(file_path, 'rb') as f:
#             data = pickle.load(f)
#         # Check if the DataFrame is not empty
#         if isinstance(data, pd.DataFrame) and not data.empty:
#             print(f"Data exists in {file_path} and has {len(data)} rows and {len(data.columns)} columns.")
#         else:
#             print(f"File {file_path} is either empty or not a DataFrame.")
#     except Exception as e:
#         print(f"Error reading {file_path}: {e}")

# # Test each .pkl file
# test_data_in_pkl(train_path)
# test_data_in_pkl(valid_path)
# test_data_in_pkl(test_path)